##Data Kemdikbud scraper
Because somehow they make it very painful to just download the data.

In [7]:
import urllib
from bs4 import BeautifulSoup
import pandas as pd

Get list of kabupaten/kota

In [162]:
master = urllib.request.urlopen("http://sekolah.data.kemdikbud.go.id")
mastersoup = BeautifulSoup(master)

In [186]:
list_daerah=[]
for opt in mastersoup.find_all('option'):
    daerah={}
    sc_code = opt.get('value')
    if sc_code:
        daerah["code"]=sc_code
        daerah["name"]=''.join(opt.findAll(text=True))
        list_daerah.append(daerah)
list_daerah = list_daerah[:-8]

Scrape marker in the response

In [36]:
f = urllib.request.urlopen("http://sekolah.data.kemdikbud.go.id/index.php/cpetasebaran/index/060000/SD") 
soup = BeautifulSoup(f)
sc = soup.find_all('script')[21].string

In [53]:
sc_c = sc.strip(' \t\n\r').split(";")
sc_arr = []
for s in sc_c:
    sc_arr.append(s.strip(' \t\n\r'))

In [159]:
sc_name=[]
sc_latlong=[]
for line in sc_arr:
    school={}
    if "NPSN" in line:
        #url
        school["url"] = line.split("=")[7][25:61]
        #NPSN
        school["npsn"]= line.split("=")[5][36:44]
        #nama
        sc_str = line.split("=")[8]
        school["name"] = sc_str[9:sc_str.find("</a")] 
        sc_name.append(school)   
    if "latLng" in line:
        latlng = line.split("ng(")[1][0:20].split(",")
        school["lat"] = latlng[0]
        school["long"] = latlng[1]
        sc_latlong.append(school)
#merge two dict list
for n, l in zip(sc_name,sc_latlong):
    n.update(l)

load into pandas dataframe

In [157]:
d = pd.DataFrame.from_dict(sc_name)

In [161]:
d.head()

,lat,long,name,npsn,url
0,5.1132410,97.0603870,SD NEGERI 19 KUTA MAKMUR,10105952,80925090-2EF5-E011-9DF3-FFD26BD6754E
1,3.5467020,97.7791500,SD NEGERI 2 LAWE DUA,10103260,B0829A90-2EF5-E011-8B89-173ABE814591
2,5.0539000,97.1653000,SD NEGERI 8 SYAMTALIRA BAYU,10101021,B0A35190-2EF5-E011-84BA-05A827682935
3,3.4200000,97.0841000,SD NEGERI 2 LABUHAN HAJI,10102961,90F19690-2EF5-E011-A8CB-BD7296C30CBD
4,4.6173000,97.8364000,SDN SARAH TEUBE,10113616,B035B1BA-23F3-E111-BCF1-2500304F5375
